In [8]:
import pandas as pd
import matplotlib.pyplot as plt

/home/jedi/anaconda3/lib/python3.9/site-packages/matplotlib_inline/config.py:66: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):


In [9]:
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import seaborn as sns

In [5]:
!pip3 install gensim
!pip3 install pyLDAvis
!pip3 install wordcloud
# !conda init bash

     |████████████████████████████████| 363 kB 229 kB/s eta 0:00:01


In [49]:
from gensim import corpora, models
import gensim
from gensim.similarities import MatrixSimilarity
from gensim.utils import SaveLoad
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from re import sub
# import pyLDAvis.gensim
from collections import Counter
from gensim.matutils import corpus2csc, sparse2full, corpus2dense
from wordcloud import WordCloud
from sklearn.utils import resample

In [10]:
df = pd.read_csv('../cleaned_tweet_data.csv')

In [11]:
df.head()

,created_at,source,original_text,polarity,subjectivity,lang,favourite_count,retweet_count,original_author,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place
0,2022-04-22 22:17:05+00:00,"<a href=""http://twitter.com/download/android"" ...",RT @NorthstarCharts: The 10-year yield is tell...,0.16,0.540000,en,188.0,43.0,davideiacovozzi,18,18,NaN,"[{'text': 'gold', 'indices': [116, 121]}, {'te...","[{'screen_name': 'NorthstarCharts', 'name': 'N...",NaN
1,2022-04-22 13:44:53+00:00,"<a href=""http://twitter.com/download/android"" ...",RT @MichaelAArouet: German 10y mortgage rate w...,0.15,0.175000,en,179.0,32.0,davideiacovozzi,18,18,NaN,[],"[{'screen_name': 'MichaelAArouet', 'name': 'Mi...",NaN
2,2022-04-22 06:10:34+00:00,"<a href=""http://twitter.com/download/android"" ...",RT @goldseek: When? https://t.co/kO2FfHKaZg,0.00,0.000000,en,193.0,26.0,davideiacovozzi,18,18,False,[],"[{'screen_name': 'goldseek', 'name': 'Peter ⚒ ...",NaN
3,2022-04-21 17:22:09+00:00,"<a href=""http://twitter.com/download/android"" ...",RT @charliebilello: The 30-year mortgage rate ...,0.00,0.183333,en,620.0,213.0,davideiacovozzi,18,18,NaN,[],"[{'screen_name': 'charliebilello', 'name': 'Ch...",NaN
4,2022-04-21 10:32:26+00:00,"<a href=""http://twitter.com/download/android"" ...",RT @biancoresearch: Rates rise until something...,-0.40,0.400000,en,1787.0,417.0,davideiacovozzi,18,18,False,[],"[{'screen_name': 'biancoresearch', 'name': 'Ji...",NaN


In [27]:
# The shape of the dataframe is 
df.shape

(15076, 15)

In [12]:
# Total nuber of rows
len(df)

15076

In [30]:
df.shape

(15076, 15)

In [34]:
df['original_text'][0]

"RT @NorthstarCharts: The 10-year yield is telling us that there's a high risk of something 'breaking' in the system #gold #silver #crypto #…"

In [14]:
# The top 5 rows of the original_text column
df['original_text'][:5]

0    RT @NorthstarCharts: The 10-year yield is tell...
1    RT @MichaelAArouet: German 10y mortgage rate w...
2          RT @goldseek: When? https://t.co/kO2FfHKaZg
3    RT @charliebilello: The 30-year mortgage rate ...
4    RT @biancoresearch: Rates rise until something...
Name: original_text, dtype: object

The data contains unwanted characters and punctuation. A standardization is required 

Functionalized text cleaning

In [24]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /home/jedi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jedi/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [18]:
#Functions for data cleaning
wnl = WordNetLemmatizer()

def removePunc(myWord):
    """Function to remove punctuation from string inputs"""
    if myWord is None:
        return myWord
    else:
        return sub('[.:;()/!&-*@$,?^\d+]','',myWord)
        
def removeAscii(myWord):
    """Function to remove ascii from string input"""
    if myWord is None:
        return myWord
    else:
        return str(sub(r'[^\x00-\x7F]+','', myWord.strip()))

def lemmatize(myWord):
    """Function to lemmatize words"""
    if myWord is None:
        return myWord
    else:
        return str(wnl.lemmatize(myWord))

def removeStopWords(myWord):
    """Function to remove stop words"""
    if myWord is None:
        return myWord
    if myWord not in str(stopwords.words('english')):
        return myWord

def removeLinkUser(myWord):
    """Function to remove web addresses and twitter handles"""
    if not myWord.startswith('@') and not myWord.startswith('http'):
        return myWord

def prepText(myWord):
    """Final text pre-processing function"""
    return removeStopWords(
        lemmatize(
            removeAscii(
                removePunc(
                    removeLinkUser(
                        myWord.lower()
                    )
                )
            )
        )
    )

<>:9: DeprecationWarning: invalid escape sequence \d
<>:9: DeprecationWarning: invalid escape sequence \d
/tmp/ipykernel_5864/880640372.py:9: DeprecationWarning: invalid escape sequence \d
  return sub('[.:;()/!&-*@$,?^\d+]','',myWord)


In [22]:
def filterTweetList(tweetList):
    """Remove stop words, lemmatize, and clean all tweets"""
    return [[prepText(word) for word
                in tweet.split()
                    if prepText(word) is not None]
                for tweet in tweetList]

In [25]:
cleanKagTweetList = filterTweetList(df['original_text'])

In [32]:
cleanKagTweetList[0]

['-year',
 'yield',
 'telling',
 'high',
 'risk',
 'something',
 'breaking',
 'system',
 '#gold',
 '#silver',
 '#crypto',
 '#']

Next, build the functions for making the dictionary, corpus, and the LDA model

In [35]:
def makeDict(myTweetList):
    """Create dictionary from list of tokenized documents"""
    return corpora.Dictionary(myTweetList)

def makeCorpus(myTweetList,myDict):
    """Create corpus from list of tokenized documents"""
    return [myDict.doc2bow(tweet) for tweet in myTweetList]

def createLDA(myCorpus, myDictionary,myTopics=50,myPasses=10,myIterations=50,myAlpha=0.001):
    """LDA model call function"""
    return models.LdaMulticore(myCorpus, id2word=myDictionary, num_topics=myTopics, passes=myPasses,
    iterations=myIterations,alpha=myAlpha)

In [36]:
"""Create model objects"""
kagDict   = makeDict(cleanKagTweetList)
kagCorpus = makeCorpus(cleanKagTweetList, kagDict)
kagLda = createLDA(kagCorpus, kagDict)

"""Save model objects"""
SaveLoad.save(kagLda,'kaggleLDAmodel')
corpora.MmCorpus.serialize('kaggleCorpus.mm', kagCorpus)
kagDict.save('kaggleDictionary.dict')

In [39]:
kagLda = SaveLoad.load('kaggleLDAmodel')
kagDict = corpora.Dictionary.load('kaggleDictionary.dict')
kagCorpus = corpora.MmCorpus('kaggleCorpus.mm')

In [43]:
!pip3 install pyLDAvis

In [55]:
import pyLDAvis

In [56]:
pyLDAvis.enable_notebook()
ldaViz = pyLDAvis.gensim_models.prepare(kagLda, kagCorpus, kagDict)

/home/jedi/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/home/jedi/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/jedi/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/jedi/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/jedi/anaconda3/lib/python3.9/site-packages/past/builtins/mis

In [57]:
ldaViz

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
10     0.041625  0.027168       1        1  4.199905
36     0.064656  0.070291       2        1  3.807640
1      0.040208  0.068681       3        1  2.850748
7      0.063221  0.023025       4        1  2.560348
5      0.019163 -0.037983       5        1  2.554778
12     0.064596 -0.034647       6        1  2.440222
23     0.043787 -0.071137       7        1  2.392229
28     0.058435 -0.013656       8        1  2.362902
4      0.057511 -0.043456       9        1  2.321083
8      0.058692  0.059878      10        1  2.307901
2     -0.037962 -0.031506      11        1  2.276218
3      0.035079 -0.053911      12        1  2.206388
27     0.071205  0.015124      13        1  2.196865
37     0.016705  0.079269      14        1  2.171313
21     0.045050 -0.029841      15        1  2.100283
17     0.027415  0.010410      16        1  2.067315
46    -0.013429 -0.015797      17        1  2.048932
41     0.015236  0.032216      18        1  2.025599
0      0.043087 -0.043688      19        1  2.014338
49    -0.131942 -0.047006      20        1  2.001666
29     0.014625 -0.023280      21        1  1.957487
18     0.016731  0.001191      22        1  1.930283
26    -0.009954 -0.091631      23        1  1.929414
31     0.044575 -0.006611      24        1  1.924924
33     0.055373  0.022920      25        1  1.915776
15     0.002706  0.047120      26        1  1.882739
25    -0.018891  0.048708      27        1  1.877221
35     0.028818 -0.081196      28        1  1.861253
22     0.021200 -0.039671      29        1  1.848341
34     0.017861 -0.011663      30        1  1.846546
16     0.007936  0.028029      31        1  1.840658
14    -0.048778 -0.147406      32        1  1.834295
20     0.033265 -0.064797      33        1  1.801643
47    -0.076848 -0.042441      34        1  1.778027
13     0.024895  0.093100      35        1  1.759969
40    -0.034530  0.037524      36        1  1.721309
45     0.056692 -0.017187      37        1  1.717820
9     -0.034727 -0.006741      38        1  1.700183
32     0.005778  0.067727      39        1  1.681723
44     0.012376  0.000090      40        1  1.675502
38    -0.016695 -0.236664      41        1  1.669267
43    -0.025523  0.042240      42        1  1.667532
19    -0.070645  0.002094      43        1  1.637568
48     0.052776  0.111956      44        1  1.609156
11    -0.336954  0.033726      45        1  1.505858
39    -0.063055 -0.005833      46        1  1.427667
6     -0.005198  0.085660      47        1  1.373200
24    -0.101373  0.014316      48        1  1.269372
30    -0.025308  0.085209      49        1  1.230795
42    -0.109464  0.090078      50        1  1.217797, topic_info=            Term        Freq       Total Category  logprob  loglift
131       follow  382.000000  382.000000  Default  30.0000  30.0000
12             %  408.000000  408.000000  Default  29.0000  29.0000
436         back  274.000000  274.000000  Default  28.0000  28.0000
20105  #srilanka  271.000000  271.000000  Default  27.0000  27.0000
598           go  241.000000  241.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
5898      island    6.613294   65.042747  Topic50  -5.4882   2.1222
3327        late    6.638014   67.373216  Topic50  -5.4845   2.0907
127          say    7.247782  308.233558  Topic50  -5.3966   0.6580
1410       never    6.454379  135.373258  Topic50  -5.5125   1.3648
29         since    6.064566   97.543688  Topic50  -5.5748   1.6303

[3044 rows x 6 columns], token_table=       Topic      Freq           Term
term                                 
23310     17  0.942943      "artists"
18499     17  0.894521  "celebrities"
23703     36  0.889175        "defend
11520     10  0.874035          "from
2653      20  0.865160            "if
...      ...       ...            ...
1070      42  0.027951              |
1070 

In [58]:
def translateLdaIdx(myLdaModel, myLdaViz):
    """Translate lda model topics to match the topics in pyLDAvis visualization"""
    ldaVizIdx = myLdaViz[0].index
    return list(ldaVizIdx)

In [59]:
newIdx = translateLdaIdx(kagLda,ldaViz)


In [60]:
def createDenseMat(myLdaModel,myCorpus,newIdx):
    """Transform corpus to dataframe with topics matching lda visualization"""
    numTopics = myLdaModel.num_topics
    myDense = corpus2dense(myLdaModel[myCorpus],numTopics)
    myDf = pd.DataFrame(myDense)
    mySortedDf = myDf.transpose()
    mySortedDf = myDf.transpose()[newIdx]
    mySortedDf.columns = ['topic' + str(i + 1) for i in range(numTopics)]
    return mySortedDf

In [63]:
kagDf = createDenseMat(kagLda,kagCorpus,newIdx)


In [61]:
def sortByTopicToIdx(cleanedTweetList,mySortedDf,myTopic,myTopicThresh=0.1):
    """Returns an index of tweets surpassing a topic value threshold"""
    myCleanArray = np.array(cleanedTweetList)
    srtIdx = list(mySortedDf[mySortedDf[myTopic]>myTopicThresh].index)
    return srtIdx

def sortTweetsByIdx(cleanedTweetList,srtIdx):
    """Returns sorted tweets as a list based on a defined sort index"""
    myCleanArray = np.array(cleanedTweetList)
    srtTweets = list(myCleanArray[srtIdx])
    return srtTweets

In [65]:
import numpy as np

In [66]:
sortedIdx = sortByTopicToIdx(cleanKagTweetList,kagDf,'topic2',myTopicThresh=0.1)


/tmp/ipykernel_5864/43916758.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  myCleanArray = np.array(cleanedTweetList)


In [68]:
fig = plt.figure(figsize=(4,4), dpi=1600)
ax = plt.subplot(111)
plt.hist(kagDf.topic2,bins=20)

ax.tick_params(labelsize=16)
ax.set_xlabel('Bad service topic value',fontsize=24,fontweight='bold')
ax.set_ylabel('Tweets (Topic 2)',fontsize=24,fontweight='bold')
plt.show()
x = fig.tight_layout()
#fig.savefig('passingFilterTweets.png',dpi=500,bbox_inches='tight')

In [69]:
def makeWordCloud(cleanedTweetList,mySortedDf,myTopic,myTopicThresh=0.1):
    """Create word cloud of tweets passing a given threshold for a given topic"""
    sortedIdx = sortByTopicToIdx(cleanedTweetList,mySortedDf,myTopic,myTopicThresh=0.1)
    mySortedTweets = sortTweetsByIdx(cleanedTweetList,sortedIdx)
    filteredWords = ' '.join([' '.join(string) for string in mySortedTweets])
    myTopicCloud = WordCloud(max_font_size=100,scale=8).generate(filteredWords)
    fig = plt.figure(figsize=(10,10), dpi=1600)
    plt.imshow(myTopicCloud)
    plt.axis("off")
    plt.show()

In [70]:
makeWordCloud(cleanKagTweetList,kagDf,'topic2',myTopicThresh=0.2)


/tmp/ipykernel_5864/43916758.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  myCleanArray = np.array(cleanedTweetList)
/tmp/ipykernel_5864/43916758.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  myCleanArray = np.array(cleanedTweetList)
